# Monthly Performance (Daily)

In [4]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import glob
import re
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

def extract_date_from_filename(filename):
    """
    Extract date from filename pattern: SA_Campaign_List_YYYYMMDD_YYYYMMDD_hash.xlsx
    Returns the first date (start date)
    """
    patterns = [
        r'SA_Campaign_List_(\d{8})_\d{8}_.*\.xlsx',  # Original pattern
        r'(\d{8})',  # Any 8-digit date
        r'(\d{2}_\d{2}_\d{4})',  # DD_MM_YYYY format
        r'(\d{4}-\d{2}-\d{2})',  # YYYY-MM-DD format
    ]
    
    basename = os.path.basename(filename)
    
    for pattern in patterns:
        match = re.search(pattern, basename)
        if match:
            date_str = match.group(1)
            try:
                if '_' in date_str:
                    return pd.to_datetime(date_str, format='%d_%m_%Y')
                elif '-' in date_str:
                    return pd.to_datetime(date_str, format='%Y-%m-%d')
                else:
                    return pd.to_datetime(date_str, format='%Y%m%d')
            except:
                continue
    
    # Fallback: use file modification date
    mod_time = os.path.getmtime(filename)
    return pd.to_datetime(datetime.fromtimestamp(mod_time).date())

def safe_clean_currency_column(column, original_column_name):
    """
    Safely remove $ symbol and convert to float, preserve original on error
    """
    if column.dtype == 'object':
        try:
            cleaned = column.astype(str).str.replace(r'[C$,]', '', regex=True)
            cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
            result = pd.to_numeric(cleaned, errors='coerce')
            
            # Check if too many values became NaN (more than 50% loss)
            valid_original = column.notna().sum()
            valid_converted = result.notna().sum()
            
            if valid_original > 0 and (valid_converted / valid_original) < 0.5:
                print(f"Warning: {original_column_name} - too many conversion failures, keeping original")
                return column
                
            return result
        except Exception as e:
            print(f"Warning: Error cleaning currency column {original_column_name}: {e}")
            return column
    return column

def safe_convert_to_float(column, original_column_name):
    """
    Safely convert object columns to float, preserve original on error
    """
    if column.dtype == 'object':
        try:
            cleaned = column.astype(str).str.replace(r'[%,]', '', regex=True)
            cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
            cleaned = cleaned.infer_objects(copy=False)
            result = pd.to_numeric(cleaned, errors='coerce')
            
            # Check if too many values became NaN (more than 50% loss)
            valid_original = column.notna().sum()
            valid_converted = result.notna().sum()
            
            if valid_original > 0 and (valid_converted / valid_original) < 0.5:
                print(f"Warning: {original_column_name} - too many conversion failures, keeping original")
                return column
                
            return result
        except Exception as e:
            print(f"Warning: Error converting float column {original_column_name}: {e}")
            return column
    return column

def safe_convert_to_int(column, original_column_name):
    """
    Safely convert object columns to int, preserve original on error
    """
    if column.dtype == 'object':
        try:
            cleaned = column.astype(str).str.replace(r'[,]', '', regex=True)
            cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
            cleaned = cleaned.infer_objects(copy=False)
            
            # Convert to float first, then to int (handling NaN values)
            float_col = pd.to_numeric(cleaned, errors='coerce')
            
            # Check if too many values became NaN (more than 50% loss)
            valid_original = column.notna().sum()
            valid_converted = float_col.notna().sum()
            
            if valid_original > 0 and (valid_converted / valid_original) < 0.5:
                print(f"Warning: {original_column_name} - too many conversion failures, keeping original")
                return column
            
            return float_col.astype('Int64')  # Nullable integer type
        except Exception as e:
            print(f"Warning: Error converting int column {original_column_name}: {e}")
            return column
    return column

def safe_extract_asin_from_portfolio(portfolio_str):
    """
    Safely extract ASIN from Portfolio string, return original if extraction fails
    """
    if pd.isna(portfolio_str) or portfolio_str == '':
        return portfolio_str  # Keep original NaN or empty
    
    try:
        portfolio_str = str(portfolio_str).strip()
        
        # Pattern 1: B + 9 alphanumeric (most common ASIN format)
        pattern1 = r'B[A-Z0-9]{9}'
        match1 = re.search(pattern1, portfolio_str.upper())
        if match1:
            return match1.group()
        
        # Pattern 2: Any 10 consecutive alphanumeric characters
        pattern2 = r'[A-Z0-9]{10}'
        match2 = re.search(pattern2, portfolio_str.upper())
        if match2:
            return match2.group()
        
        # Pattern 3: Extract first 10 alphanumeric characters
        clean_str = re.sub(r'[^A-Za-z0-9]', '', portfolio_str)
        if len(clean_str) >= 10:
            return clean_str[:10].upper()
        
        # If no valid ASIN found, return original value
        return portfolio_str
    except Exception as e:
        print(f"Warning: Error extracting ASIN from '{portfolio_str}': {e}")
        return portfolio_str

def safe_normalize_campaign_types(text):
    """
    Safely normalize campaign type keywords, preserve original on error
    """
    if pd.isna(text) or text == '':
        return text
    
    try:
        text = str(text)
        
        normalizations = {
            'sponsoredBrands': 'SB',
            'sponsoredDisplay': 'SD', 
            'sponsoredProducts': 'SP',
            'sponsoredbrands': 'SB',
            'sponsoreddisplay': 'SD',
            'sponsoredproducts': 'SP',
            'Sponsored Brands': 'SB',
            'Sponsored Display': 'SD',
            'Sponsored Products': 'SP'
        }
        
        for original, normalized in normalizations.items():
            text = text.replace(original, normalized)
        
        return text
    except Exception as e:
        print(f"Warning: Error normalizing campaign type '{text}': {e}")
        return text

def process_single_xlsx(file_path):
    """
    Process a single XLSX file with data preservation safeguards
    """
    try:
        print(f"Processing: {os.path.basename(file_path)}")
        
        # Read Excel file
        df = pd.read_excel(file_path)
        original_shape = df.shape
        print(f"  - Original shape: {original_shape}")
        
        # Remove completely empty rows and columns (but be conservative)
        df_cleaned = df.dropna(axis=0, how='all')  # Remove empty rows
        df_cleaned = df_cleaned.dropna(axis=1, how='all')  # Remove empty columns
        
        # Check if we lost too many rows (safety check)
        if len(df_cleaned) < len(df) * 0.9:  # If we lose more than 10% of rows
            print(f"  Warning: Potential data loss in row cleaning, using original data")
            df_cleaned = df
        
        df = df_cleaned
        print(f"  - After cleaning: {df.shape}")
        
        # Clean column names safely
        original_columns = df.columns.tolist()
        try:
            df.columns = [str(col).strip() for col in df.columns]
        except Exception as e:
            print(f"  Warning: Error cleaning column names: {e}")
            df.columns = original_columns
        
        # Extract date from filename
        date_extracted = extract_date_from_filename(file_path)
        print(f"  - Extracted date: {date_extracted}")
        
        # Drop specified columns if they exist (but safely)
        columns_to_drop = ['Profile', 'Labels', 'Budget group']
        existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
        if existing_columns_to_drop:
            try:
                df = df.drop(columns=existing_columns_to_drop)
                print(f"  - Dropped columns: {existing_columns_to_drop}")
            except Exception as e:
                print(f"  Warning: Error dropping columns: {e}")
        
        # Create ASIN column from Portfolio (safely)
        asin_values = None
        if 'Portfolio' in df.columns:
            try:
                asin_values = df['Portfolio'].apply(safe_extract_asin_from_portfolio)
            except Exception as e:
                print(f"  Warning: Error creating ASIN column: {e}")
                asin_values = df['Portfolio']  # Use original Portfolio column
        else:
            # If no Portfolio column, create empty ASIN column
            asin_values = [None] * len(df)
        
        # Create Date column
        date_values = [date_extracted] * len(df)
        
        # Normalize campaign types safely
        if 'Campaign type' in df.columns:
            try:
                df['Campaign type'] = df['Campaign type'].apply(safe_normalize_campaign_types)
            except Exception as e:
                print(f"  Warning: Error normalizing campaign types: {e}")
        
        # Clean currency columns safely
        currency_columns = ['Daily Budget', 'Current Budget']
        for col in currency_columns:
            if col in df.columns:
                df[col] = safe_clean_currency_column(df[col], col)
        
        # Convert float columns safely
        float_columns = ['Avg.time in Budget', 'Top-of-search IS', 'CPC', 'CVR', 'ACOS', 'ROAS', 'CPA', 'CTR']
        for col in float_columns:
            if col in df.columns:
                df[col] = safe_convert_to_float(df[col], col)
        
        # Convert int columns safely
        int_columns = ['Orders Other SKU', 'Units Other SKU', 'Orders Same SKU', 'Units Same SKU', 
                      'Impressions', 'Clicks', 'Orders', 'Units']
        for col in int_columns:
            if col in df.columns:
                df[col] = safe_convert_to_int(df[col], col)
        
        # Define exact required columns only
        required_columns = [
            'ASIN', 'Date', 'Campaign type', 'Campaign', 'Status', 'Country', 'Portfolio',
            'Daily Budget', 'Bidding Strategy', 'Top-of-search IS', 'Avg.time in Budget',
            'Impressions', 'Clicks', 'CTR', 'Spend', 'CPC', 'Orders', 'Sales', 'Units',
            'CVR', 'ACOS', 'ROAS', 'CPA', 'Sales Same SKU', 'Sales Other SKU',
            'Orders Same SKU', 'Orders Other SKU', 'Units Same SKU', 'Units Other SKU'
        ]
        
        # Create new DataFrame with required columns (preserve all data)
        ordered_df = pd.DataFrame()
        
        # Add ASIN as first column
        ordered_df['ASIN'] = asin_values
        
        # Add Date as second column
        ordered_df['Date'] = date_values
        
        # Add remaining required columns in specified order
        for col in required_columns[2:]:  # Skip ASIN and Date since already added
            if col in df.columns:
                ordered_df[col] = df[col]
            else:
                ordered_df[col] = np.nan  # Add missing columns with NaN
                print(f"  - Missing column filled with NaN: {col}")
        
        # Add any additional columns that weren't in the required list (preserve extra data)
        extra_columns = [col for col in df.columns if col not in required_columns and col not in ['ASIN', 'Date']]
        for col in extra_columns:
            new_col_name = f"Extra_{col}"  # Prefix to identify extra columns
            ordered_df[new_col_name] = df[col]
            print(f"  - Preserved extra column as: {new_col_name}")
        
        final_shape = ordered_df.shape
        print(f"  - Final shape: {final_shape}")
        print(f"  - Data preservation check: {len(ordered_df)} rows maintained")
        
        return ordered_df
        
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        print("Attempting to return minimal processed data to avoid total loss...")
        
        # Last resort: return basic DataFrame with original data
        try:
            basic_df = pd.read_excel(file_path)
            # Just add Date column and return
            basic_df['Date'] = extract_date_from_filename(file_path)
            return basic_df
        except:
            return None

def process_folder(folder_path):
    """
    Process all XLSX files in a folder with data preservation
    """
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return pd.DataFrame()
    
    # Find all XLSX files
    xlsx_pattern = os.path.join(folder_path, "*.xlsx")
    xlsx_files = glob.glob(xlsx_pattern)
    
    # Filter out temporary Excel files
    xlsx_files = [f for f in xlsx_files if not os.path.basename(f).startswith('~')]
    
    if not xlsx_files:
        print(f"No Excel files found in {folder_path}")
        return pd.DataFrame()
    
    print(f"Found {len(xlsx_files)} Excel files in {folder_path}")
    
    # Process each file and collect DataFrames
    dataframes = []
    successful_files = []
    failed_files = []
    
    for file_path in sorted(xlsx_files):  # Sort for consistent order
        df = process_single_xlsx(file_path)
        if df is not None and not df.empty:
            dataframes.append(df)
            successful_files.append(os.path.basename(file_path))
        else:
            failed_files.append(os.path.basename(file_path))
    
    # Report processing results
    print(f"\nProcessing Summary:")
    print(f"  - Successful: {len(successful_files)} files")
    print(f"  - Failed: {len(failed_files)} files")
    
    if failed_files:
        print(f"  - Failed files: {failed_files}")
    
    # Combine all DataFrames safely
    if dataframes:
        try:
            combined_df = pd.concat(dataframes, ignore_index=True, sort=False)
            print(f"Combined {len(successful_files)} files into {len(combined_df)} total rows")
            return combined_df
        except Exception as e:
            print(f"Error combining DataFrames: {e}")
            # Try to return the largest DataFrame as fallback
            if dataframes:
                largest_df = max(dataframes, key=len)
                print(f"Returning largest individual DataFrame with {len(largest_df)} rows")
                return largest_df
    
    print(f"No valid data found in {folder_path}")
    return pd.DataFrame()

def main():
    """
    Main function to process Ads folders with data preservation
    """
    # Define folder paths - FIXED: Variable naming issue
    base_path = "C:/Users/admin1/Desktop/Performance-Tracking/Ads-XNurta"
    ads_m7_path = os.path.join(base_path, "H2_2025_US", "Tháng 7")
    ads_m8_path = os.path.join(base_path, "H2_2025_US", "Tháng 8")
    ads_m9_path = os.path.join(base_path, "H2_2025_US", "Tháng 9")  # FIXED: Was overwriting ads_m8_path
    
    # Check if folders exist and prepare processing list
    folders_to_process = []
    
    if os.path.exists(ads_m7_path):
        folders_to_process.append(("Tháng 7", ads_m7_path))
    else:
        print(f"Warning: {ads_m7_path} not found")
    
    if os.path.exists(ads_m8_path):
        folders_to_process.append(("Tháng 8", ads_m8_path))
    else:
        print(f"Warning: {ads_m8_path} not found")
        
    if os.path.exists(ads_m9_path):
        folders_to_process.append(("Tháng 9", ads_m9_path))
    else:
        print(f"Warning: {ads_m9_path} not found")
    
    if not folders_to_process:
        print("No valid folders found. Please check your paths.")
        return pd.DataFrame()
    
    print(f"Found {len(folders_to_process)} folders to process")
    
    # Process each folder
    all_dataframes = []
    processing_summary = []
    
    for folder_name, folder_path in folders_to_process:
        print(f"\n=== Processing {folder_name} ===")
        df = process_folder(folder_path)
        if not df.empty:
            all_dataframes.append(df)
            processing_summary.append(f"{folder_name}: {len(df)} rows")
        else:
            processing_summary.append(f"{folder_name}: No data")
    
    # Combine all data from folders
    if all_dataframes:
        print(f"\n=== Combining Data from All Folders ===")
        try:
            final_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
            print(f"Successfully combined data from {len(all_dataframes)} folders")
            
            # Safe duplicate removal (only if key columns exist)
            if 'ASIN' in final_df.columns and 'Campaign' in final_df.columns and 'Date' in final_df.columns:
                original_rows = len(final_df)
                final_df = final_df.drop_duplicates(subset=['ASIN', 'Date', 'Campaign'], keep='last')
                removed_duplicates = original_rows - len(final_df)
                print(f"Removed {removed_duplicates} duplicate rows")
            
            # Safe sorting
            try:
                final_df = final_df.sort_values(['ASIN', 'Date'], na_position='last')
                final_df = final_df.reset_index(drop=True)
            except Exception as e:
                print(f"Warning: Error sorting data: {e}")
            
            print(f"\n=== Final Results ===")
            print(f"Total rows: {len(final_df)}")
            print(f"Total columns: {len(final_df.columns)}")
            
            for summary in processing_summary:
                print(f"  - {summary}")
            
            if 'Date' in final_df.columns:
                try:
                    date_min = final_df['Date'].min()
                    date_max = final_df['Date'].max()
                    print(f"Date range: {date_min} to {date_max}")
                except:
                    print("Date range: Unable to determine")
            
            if 'ASIN' in final_df.columns:
                try:
                    unique_asins = final_df['ASIN'].nunique()
                    print(f"Unique ASINs: {unique_asins}")
                except:
                    print("Unique ASINs: Unable to determine")
            
            return final_df
            
        except Exception as e:
            print(f"Error combining data: {e}")
            # Return the largest DataFrame as fallback
            if all_dataframes:
                largest_df = max(all_dataframes, key=len)
                print(f"Returning largest DataFrame with {len(largest_df)} rows as fallback")
                return largest_df
    
    print("No data to process.")
    return pd.DataFrame()

# Keep all the other utility functions unchanged
def save_to_excel(df, filename):
    """Save DataFrame to Excel with proper formatting"""
    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Combined_Ads_Data', index=False)
            print(f"Data successfully saved to: {filename}")
    except Exception as e:
        print(f"Error saving to Excel: {e}")

def display_sample(df, rows=3):
    """Display sample data with proper formatting"""
    try:
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', 20)
        print(df.head(rows).to_string(index=False))
        pd.reset_option('display.max_columns')
        pd.reset_option('display.width')
        pd.reset_option('display.max_colwidth')
    except Exception as e:
        print(f"Error displaying sample: {e}")

if __name__ == "__main__":
    # Run the main processing
    print("Starting Amazon Ads Data Processing with Data Preservation...")
    print("=" * 60)
    
    result_df = main()
    
    if not result_df.empty:
        try:
            # Save combined data
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            output_filename = f"Combined_Ads_Data_Safe_{timestamp}.xlsx"
            save_to_excel(result_df, output_filename)
            
            # Display sample
            print(f"\nSample data (first 3 rows):")
            display_sample(result_df)
            
            print(f"\n=== Column List ===")
            for i, col in enumerate(result_df.columns, 1):
                print(f"{i:2d}. {col}")
                
        except Exception as e:
            print(f"Error in final processing: {e}")
    
    print("\n" + "=" * 60)
    print("Processing completed with data preservation safeguards!")

Starting Amazon Ads Data Processing with Data Preservation...
Found 3 folders to process

=== Processing Tháng 7 ===
Found 31 Excel files in C:/Users/admin1/Desktop/Performance-Tracking/Ads-XNurta\H2_2025_US\Tháng 7
Processing: SA_Campaign_List_20250701_20250701_CNW4yt.xlsx
  - Original shape: (675, 33)
  - After cleaning: (675, 32)
  - Extracted date: 2025-07-01 00:00:00
  - Dropped columns: ['Profile', 'Labels']
  - Preserved extra column as: Extra_Target type
  - Preserved extra column as: Extra_Current Budget
  - Preserved extra column as: Extra_SP Off-site Ads Strategy
  - Final shape: (675, 32)
  - Data preservation check: 675 rows maintained
Processing: SA_Campaign_List_20250702_20250702_2akRmQ.xlsx
  - Original shape: (551, 33)
  - After cleaning: (551, 32)
  - Extracted date: 2025-07-02 00:00:00
  - Dropped columns: ['Profile', 'Labels']
  - Preserved extra column as: Extra_Target type
  - Preserved extra column as: Extra_Current Budget
  - Preserved extra column as: Extra_SP 

In [5]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("c:/Users/admin1/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1lZ4dsi94HaeWshsEizKTyNHeOOG0tpLJhzL9pMxvd6k"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("Raw_XN_Q3_2025_US")

set_with_dataframe(sheet1, result_df)

# Append Selected Month

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import glob
import re
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

def extract_date_from_filename(filename):
    """
    Extract date from filename pattern: SA_Campaign_List_YYYYMMDD_YYYYMMDD_hash.xlsx
    Returns the first date (start date)
    """
    patterns = [
        r'SA_Campaign_List_(\d{8})_\d{8}_.*\.xlsx',  # Original pattern
        r'(\d{8})',  # Any 8-digit date
        r'(\d{2}_\d{2}_\d{4})',  # DD_MM_YYYY format
        r'(\d{4}-\d{2}-\d{2})',  # YYYY-MM-DD format
    ]
    
    basename = os.path.basename(filename)
    
    for pattern in patterns:
        match = re.search(pattern, basename)
        if match:
            date_str = match.group(1)
            try:
                if '_' in date_str:
                    return pd.to_datetime(date_str, format='%d_%m_%Y')
                elif '-' in date_str:
                    return pd.to_datetime(date_str, format='%Y-%m-%d')
                else:
                    return pd.to_datetime(date_str, format='%Y%m%d')
            except:
                continue
    
    # Fallback: use file modification date
    mod_time = os.path.getmtime(filename)
    return pd.to_datetime(datetime.fromtimestamp(mod_time).date())

def safe_clean_currency_column(column, original_column_name):
    """
    Safely remove $ symbol and convert to float, preserve original on error
    """
    if column.dtype == 'object':
        try:
            cleaned = column.astype(str).str.replace(r'[C$,]', '', regex=True)
            cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
            result = pd.to_numeric(cleaned, errors='coerce')
            
            # Check if too many values became NaN (more than 50% loss)
            valid_original = column.notna().sum()
            valid_converted = result.notna().sum()
            
            if valid_original > 0 and (valid_converted / valid_original) < 0.5:
                print(f"Warning: {original_column_name} - too many conversion failures, keeping original")
                return column
                
            return result
        except Exception as e:
            print(f"Warning: Error cleaning currency column {original_column_name}: {e}")
            return column
    return column

def safe_convert_to_float(column, original_column_name):
    """
    Safely convert object columns to float, preserve original on error
    """
    if column.dtype == 'object':
        try:
            cleaned = column.astype(str).str.replace(r'[%,]', '', regex=True)
            cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
            cleaned = cleaned.infer_objects(copy=False)
            result = pd.to_numeric(cleaned, errors='coerce')
            
            # Check if too many values became NaN (more than 50% loss)
            valid_original = column.notna().sum()
            valid_converted = result.notna().sum()
            
            if valid_original > 0 and (valid_converted / valid_original) < 0.5:
                print(f"Warning: {original_column_name} - too many conversion failures, keeping original")
                return column
                
            return result
        except Exception as e:
            print(f"Warning: Error converting float column {original_column_name}: {e}")
            return column
    return column

def safe_convert_to_int(column, original_column_name):
    """
    Safely convert object columns to int, preserve original on error
    """
    if column.dtype == 'object':
        try:
            cleaned = column.astype(str).str.replace(r'[,]', '', regex=True)
            cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
            cleaned = cleaned.infer_objects(copy=False)
            
            # Convert to float first, then to int (handling NaN values)
            float_col = pd.to_numeric(cleaned, errors='coerce')
            
            # Check if too many values became NaN (more than 50% loss)
            valid_original = column.notna().sum()
            valid_converted = float_col.notna().sum()
            
            if valid_original > 0 and (valid_converted / valid_original) < 0.5:
                print(f"Warning: {original_column_name} - too many conversion failures, keeping original")
                return column
            
            return float_col.astype('Int64')  # Nullable integer type
        except Exception as e:
            print(f"Warning: Error converting int column {original_column_name}: {e}")
            return column
    return column

def safe_extract_asin_from_portfolio(portfolio_str):
    """
    Safely extract ASIN from Portfolio string, return original if extraction fails
    """
    if pd.isna(portfolio_str) or portfolio_str == '':
        return portfolio_str  # Keep original NaN or empty
    
    try:
        portfolio_str = str(portfolio_str).strip()
        
        # Pattern 1: B + 9 alphanumeric (most common ASIN format)
        pattern1 = r'B[A-Z0-9]{9}'
        match1 = re.search(pattern1, portfolio_str.upper())
        if match1:
            return match1.group()
        
        # Pattern 2: Any 10 consecutive alphanumeric characters
        pattern2 = r'[A-Z0-9]{10}'
        match2 = re.search(pattern2, portfolio_str.upper())
        if match2:
            return match2.group()
        
        # Pattern 3: Extract first 10 alphanumeric characters
        clean_str = re.sub(r'[^A-Za-z0-9]', '', portfolio_str)
        if len(clean_str) >= 10:
            return clean_str[:10].upper()
        
        # If no valid ASIN found, return original value
        return portfolio_str
    except Exception as e:
        print(f"Warning: Error extracting ASIN from '{portfolio_str}': {e}")
        return portfolio_str

def safe_normalize_campaign_types(text):
    """
    Safely normalize campaign type keywords, preserve original on error
    """
    if pd.isna(text) or text == '':
        return text
    
    try:
        text = str(text)
        
        normalizations = {
            'sponsoredBrands': 'SB',
            'sponsoredDisplay': 'SD', 
            'sponsoredProducts': 'SP',
            'sponsoredbrands': 'SB',
            'sponsoreddisplay': 'SD',
            'sponsoredproducts': 'SP',
            'Sponsored Brands': 'SB',
            'Sponsored Display': 'SD',
            'Sponsored Products': 'SP'
        }
        
        for original, normalized in normalizations.items():
            text = text.replace(original, normalized)
        
        return text
    except Exception as e:
        print(f"Warning: Error normalizing campaign type '{text}': {e}")
        return text

def process_single_xlsx(file_path):
    """
    Process a single XLSX file with data preservation safeguards - MODIFIED to exclude specific extra columns
    """
    try:
        print(f"Processing: {os.path.basename(file_path)}")
        
        # Read Excel file
        df = pd.read_excel(file_path)
        original_shape = df.shape
        print(f"  - Original shape: {original_shape}")
        
        # Remove completely empty rows and columns (but be conservative)
        df_cleaned = df.dropna(axis=0, how='all')  # Remove empty rows
        df_cleaned = df_cleaned.dropna(axis=1, how='all')  # Remove empty columns
        
        # Check if we lost too many rows (safety check)
        if len(df_cleaned) < len(df) * 0.9:  # If we lose more than 10% of rows
            print(f"  Warning: Potential data loss in row cleaning, using original data")
            df_cleaned = df
        
        df = df_cleaned
        print(f"  - After cleaning: {df.shape}")
        
        # Clean column names safely
        original_columns = df.columns.tolist()
        try:
            df.columns = [str(col).strip() for col in df.columns]
        except Exception as e:
            print(f"  Warning: Error cleaning column names: {e}")
            df.columns = original_columns
        
        # Extract date from filename
        date_extracted = extract_date_from_filename(file_path)
        print(f"  - Extracted date: {date_extracted}")
        
        # Drop specified columns if they exist (but safely)
        columns_to_drop = ['Profile', 'Labels', 'Budget group']
        existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
        if existing_columns_to_drop:
            try:
                df = df.drop(columns=existing_columns_to_drop)
                print(f"  - Dropped columns: {existing_columns_to_drop}")
            except Exception as e:
                print(f"  Warning: Error dropping columns: {e}")
        
        # Create ASIN column from Portfolio (safely)
        asin_values = None
        if 'Portfolio' in df.columns:
            try:
                asin_values = df['Portfolio'].apply(safe_extract_asin_from_portfolio)
            except Exception as e:
                print(f"  Warning: Error creating ASIN column: {e}")
                asin_values = df['Portfolio']  # Use original Portfolio column
        else:
            # If no Portfolio column, create empty ASIN column
            asin_values = [None] * len(df)
        
        # Create Date column
        date_values = [date_extracted] * len(df)
        
        # Normalize campaign types safely
        if 'Campaign type' in df.columns:
            try:
                df['Campaign type'] = df['Campaign type'].apply(safe_normalize_campaign_types)
            except Exception as e:
                print(f"  Warning: Error normalizing campaign types: {e}")
        
        # Clean currency columns safely
        currency_columns = ['Daily Budget', 'Current Budget']
        for col in currency_columns:
            if col in df.columns:
                df[col] = safe_clean_currency_column(df[col], col)
        
        # Convert float columns safely
        float_columns = ['Avg.time in Budget', 'Top-of-search IS', 'CPC', 'CVR', 'ACOS', 'ROAS', 'CPA', 'CTR']
        for col in float_columns:
            if col in df.columns:
                df[col] = safe_convert_to_float(df[col], col)
        
        # Convert int columns safely
        int_columns = ['Orders Other SKU', 'Units Other SKU', 'Orders Same SKU', 'Units Same SKU', 
                      'Impressions', 'Clicks', 'Orders', 'Units']
        for col in int_columns:
            if col in df.columns:
                df[col] = safe_convert_to_int(df[col], col)
        
        # Define exact required columns only
        required_columns = [
            'ASIN', 'Date', 'Campaign type', 'Campaign', 'Status', 'Country', 'Portfolio',
            'Daily Budget', 'Bidding Strategy', 'Top-of-search IS', 'Avg.time in Budget',
            'Impressions', 'Clicks', 'CTR', 'Spend', 'CPC', 'Orders', 'Sales', 'Units',
            'CVR', 'ACOS', 'ROAS', 'CPA', 'Sales Same SKU', 'Sales Other SKU',
            'Orders Same SKU', 'Orders Other SKU', 'Units Same SKU', 'Units Other SKU'
        ]
        
        # MODIFIED: Define columns to exclude from extra columns
        excluded_extra_columns = ['Target type', 'Current Budget', 'SP Off-site Ads Strategy']
        
        # Create new DataFrame with required columns (preserve all data)
        ordered_df = pd.DataFrame()
        
        # Add ASIN as first column
        ordered_df['ASIN'] = asin_values
        
        # Add Date as second column
        ordered_df['Date'] = date_values
        
        # Add remaining required columns in specified order
        for col in required_columns[2:]:  # Skip ASIN and Date since already added
            if col in df.columns:
                ordered_df[col] = df[col]
            else:
                ordered_df[col] = np.nan  # Add missing columns with NaN
                print(f"  - Missing column filled with NaN: {col}")
        
        # MODIFIED: Add any additional columns that weren't in the required list (preserve extra data, but exclude specified columns)
        extra_columns = [col for col in df.columns 
                        if col not in required_columns 
                        and col not in ['ASIN', 'Date'] 
                        and col not in excluded_extra_columns]  # NEW: Exclude unwanted columns
        
        for col in extra_columns:
            new_col_name = f"Extra_{col}"  # Prefix to identify extra columns
            ordered_df[new_col_name] = df[col]
            print(f"  - Preserved extra column as: {new_col_name}")
        
        # Print excluded columns for transparency
        excluded_found = [col for col in excluded_extra_columns if col in df.columns]
        if excluded_found:
            print(f"  - Excluded extra columns: {excluded_found}")
        
        final_shape = ordered_df.shape
        print(f"  - Final shape: {final_shape}")
        print(f"  - Data preservation check: {len(ordered_df)} rows maintained")
        
        return ordered_df
        
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        print("Attempting to return minimal processed data to avoid total loss...")
        
        # Last resort: return basic DataFrame with original data
        try:
            basic_df = pd.read_excel(file_path)
            # Just add Date column and return
            basic_df['Date'] = extract_date_from_filename(file_path)
            return basic_df
        except:
            return None

def process_folder(folder_path):
    """
    Process all XLSX files in a folder with data preservation
    """
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return pd.DataFrame()
    
    # Find all XLSX files
    xlsx_pattern = os.path.join(folder_path, "*.xlsx")
    xlsx_files = glob.glob(xlsx_pattern)
    
    # Filter out temporary Excel files
    xlsx_files = [f for f in xlsx_files if not os.path.basename(f).startswith('~')]
    
    if not xlsx_files:
        print(f"No Excel files found in {folder_path}")
        return pd.DataFrame()
    
    print(f"Found {len(xlsx_files)} Excel files in {folder_path}")
    
    # Process each file and collect DataFrames
    dataframes = []
    successful_files = []
    failed_files = []
    
    for file_path in sorted(xlsx_files):  # Sort for consistent order
        df = process_single_xlsx(file_path)
        if df is not None and not df.empty:
            dataframes.append(df)
            successful_files.append(os.path.basename(file_path))
        else:
            failed_files.append(os.path.basename(file_path))
    
    # Report processing results
    print(f"\nProcessing Summary:")
    print(f"  - Successful: {len(successful_files)} files")
    print(f"  - Failed: {len(failed_files)} files")
    
    if failed_files:
        print(f"  - Failed files: {failed_files}")
    
    # Combine all DataFrames safely
    if dataframes:
        try:
            combined_df = pd.concat(dataframes, ignore_index=True, sort=False)
            print(f"Combined {len(successful_files)} files into {len(combined_df)} total rows")
            return combined_df
        except Exception as e:
            print(f"Error combining DataFrames: {e}")
            # Try to return the largest DataFrame as fallback
            if dataframes:
                largest_df = max(dataframes, key=len)
                print(f"Returning largest individual DataFrame with {len(largest_df)} rows")
                return largest_df
    
    print(f"No valid data found in {folder_path}")
    return pd.DataFrame()

def main_month9_only():
    """
    MODIFIED: Main function to process ONLY Month 9 data
    """
    # Define folder paths - ONLY Month 9
    base_path = "C:/Users/admin1/Desktop/Performance-Tracking/Ads-XNurta"
    ads_m9_path = os.path.join(base_path, "H2_2025_CA", "Tháng 9")
    
    # Check if Month 9 folder exists
    if not os.path.exists(ads_m9_path):
        print(f"Warning: {ads_m9_path} not found")
        return pd.DataFrame()
    
    print(f"Processing only Month 9 data from: {ads_m9_path}")
    
    # Process Month 9 folder only
    print(f"\n=== Processing Tháng 9 ===")
    df = process_folder(ads_m9_path)
    
    if df.empty:
        print("No data found for Month 9")
        return pd.DataFrame()
    
    # Safe duplicate removal (only if key columns exist)
    if 'ASIN' in df.columns and 'Campaign' in df.columns and 'Date' in df.columns:
        original_rows = len(df)
        df = df.drop_duplicates(subset=['ASIN', 'Date', 'Campaign'], keep='last')
        removed_duplicates = original_rows - len(df)
        print(f"Removed {removed_duplicates} duplicate rows")
    
    # Safe sorting
    try:
        df = df.sort_values(['ASIN', 'Date'], na_position='last')
        df = df.reset_index(drop=True)
    except Exception as e:
        print(f"Warning: Error sorting data: {e}")
    
    print(f"\n=== Month 9 Results ===")
    print(f"Total rows: {len(df)}")
    print(f"Total columns: {len(df.columns)}")
    
    if 'Date' in df.columns:
        try:
            date_min = df['Date'].min()
            date_max = df['Date'].max()
            print(f"Date range: {date_min} to {date_max}")
        except:
            print("Date range: Unable to determine")
    
    if 'ASIN' in df.columns:
        try:
            unique_asins = df['ASIN'].nunique()
            print(f"Unique ASINs: {unique_asins}")
        except:
            print("Unique ASINs: Unable to determine")
    
    return df

def save_to_excel(df, filename):
    """Save DataFrame to Excel with proper formatting"""
    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Month9_Ads_Data', index=False)
            print(f"Data successfully saved to: {filename}")
    except Exception as e:
        print(f"Error saving to Excel: {e}")

def display_sample(df, rows=3):
    """Display sample data with proper formatting"""
    try:
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', 20)
        print(df.head(rows).to_string(index=False))
        pd.reset_option('display.max_columns')
        pd.reset_option('display.width')
        pd.reset_option('display.max_colwidth')
    except Exception as e:
        print(f"Error displaying sample: {e}")

def get_existing_row_count(sheet):
    """
    Get the number of existing rows in the Google Sheet (excluding header)
    """
    try:
        all_values = sheet.get_all_values()
        # Return count minus header row (assuming first row is header)
        return len(all_values) - 1 if len(all_values) > 0 else 0
    except Exception as e:
        print(f"Error getting row count: {e}")
        return 0

def append_to_google_sheet(sheet, new_df, existing_rows):
    """
    Append new data to Google Sheet starting from the next available row
    """
    try:
        if new_df.empty:
            print("No data to append")
            return
        
        # Calculate the starting row for appending (existing_rows + 2 to account for header)
        start_row = existing_rows + 2
        
        print(f"Appending {len(new_df)} rows starting from row {start_row}")
        
        # Convert DataFrame to list of lists for batch update
        values = new_df.values.tolist()
        
        # Prepare the range for batch update
        end_col_letter = chr(ord('A') + len(new_df.columns) - 1)  # Convert to column letter
        range_name = f"A{start_row}:{end_col_letter}{start_row + len(values) - 1}"
        
        print(f"Updating range: {range_name}")
        
        # Batch update the sheet
        sheet.update(range_name, values)
        print(f"Successfully appended {len(new_df)} rows to Google Sheet")
        
    except Exception as e:
        print(f"Error appending to Google Sheet: {e}")
        # Fallback: try using set_with_dataframe but with an offset
        try:
            print("Trying fallback method...")
            # This will overwrite, but we'll note the issue
            set_with_dataframe(sheet, new_df, row=start_row, include_column_header=False)
            print("Fallback append completed")
        except Exception as e2:
            print(f"Fallback method also failed: {e2}")

if __name__ == "__main__":
    # MODIFIED: Run only Month 9 processing and append to Google Sheet
    print("Starting Amazon Ads Data Processing - Month 9 Only with Append...")
    print("=" * 60)
    
    # Process Month 9 data
    result_df = main_month9_only()
    
    if not result_df.empty:
        try:
            # Save to local Excel file
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            output_filename = f"Month9_Ads_Data_{timestamp}.xlsx"
            save_to_excel(result_df, output_filename)
            
            # Display sample
            print(f"\nSample data (first 3 rows):")
            display_sample(result_df)
            
            print(f"\n=== Column List ===")
            for i, col in enumerate(result_df.columns, 1):
                print(f"{i:2d}. {col}")
            
            # MODIFIED: Connect to Google Sheets and append data
            print(f"\n=== Uploading to Google Sheet ===")
            
            try:
                scopes = ["https://www.googleapis.com/auth/spreadsheets", 
                          "https://www.googleapis.com/auth/drive"]
                creds = Credentials.from_service_account_file("c:/Users/admin1/Downloads/new_credential.json", scopes=scopes)
                client = gspread.authorize(creds)

                # Open Google Sheet
                sheet_id = "1lZ4dsi94HaeWshsEizKTyNHeOOG0tpLJhzL9pMxvd6k"
                spreadsheet = client.open_by_key(sheet_id)
                sheet1 = spreadsheet.worksheet("Raw_XN_Q3_2025_CA")
                
                # Get existing row count
                existing_rows = get_existing_row_count(sheet1)
                print(f"Existing rows in sheet: {existing_rows}")
                
                # Append new data
                append_to_google_sheet(sheet1, result_df, existing_rows)
                
                print("Google Sheet update completed successfully!")
                
            except Exception as e:
                print(f"Error updating Google Sheet: {e}")
                print("Local Excel file has been saved successfully.")
                
        except Exception as e:
            print(f"Error in final processing: {e}")
    else:
        print("No data processed for Month 9")
    
    print("\n" + "=" * 60)
    print("Month 9 processing completed!")

Starting Amazon Ads Data Processing - Month 9 Only with Append...
Processing only Month 9 data from: C:/Users/admin1/Desktop/Performance-Tracking/Ads-XNurta\H2_2025_CA\Tháng 9

=== Processing Tháng 9 ===
Found 9 Excel files in C:/Users/admin1/Desktop/Performance-Tracking/Ads-XNurta\H2_2025_CA\Tháng 9
Processing: SA_Campaign_List_20250901_20250901_Puk9bN.xlsx
  - Original shape: (41, 33)
  - After cleaning: (41, 32)
  - Extracted date: 2025-09-01 00:00:00
  - Dropped columns: ['Profile', 'Labels']
  - Excluded extra columns: ['Target type', 'Current Budget', 'SP Off-site Ads Strategy']
  - Final shape: (41, 29)
  - Data preservation check: 41 rows maintained
Processing: SA_Campaign_List_20250902_20250902_c6f3TZ.xlsx


C:\Users\admin1\AppData\Local\Temp\ipykernel_23940\4164627072.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
C:\Users\admin1\AppData\Local\Temp\ipykernel_23940\4164627072.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
C:\Users\admin1\AppData\Local\Temp\ipykernel_23940\4164627072.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future versi

  - Original shape: (41, 33)
  - After cleaning: (41, 32)
  - Extracted date: 2025-09-02 00:00:00
  - Dropped columns: ['Profile', 'Labels']
  - Excluded extra columns: ['Target type', 'Current Budget', 'SP Off-site Ads Strategy']
  - Final shape: (41, 29)
  - Data preservation check: 41 rows maintained
Processing: SA_Campaign_List_20250903_20250903_fTfLbo.xlsx
  - Original shape: (41, 33)
  - After cleaning: (41, 32)
  - Extracted date: 2025-09-03 00:00:00
  - Dropped columns: ['Profile', 'Labels']
  - Excluded extra columns: ['Target type', 'Current Budget', 'SP Off-site Ads Strategy']
  - Final shape: (41, 29)
  - Data preservation check: 41 rows maintained
Processing: SA_Campaign_List_20250904_20250904_DMIVSS.xlsx
  - Original shape: (44, 33)
  - After cleaning: (44, 32)
  - Extracted date: 2025-09-04 00:00:00
  - Dropped columns: ['Profile', 'Labels']
  - Excluded extra columns: ['Target type', 'Current Budget', 'SP Off-site Ads Strategy']
  - Final shape: (44, 29)
  - Data preser

C:\Users\admin1\AppData\Local\Temp\ipykernel_23940\4164627072.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
C:\Users\admin1\AppData\Local\Temp\ipykernel_23940\4164627072.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
C:\Users\admin1\AppData\Local\Temp\ipykernel_23940\4164627072.py:100: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future versi

  - Original shape: (45, 33)
  - After cleaning: (45, 32)
  - Extracted date: 2025-09-07 00:00:00
  - Dropped columns: ['Profile', 'Labels']
  - Excluded extra columns: ['Target type', 'Current Budget', 'SP Off-site Ads Strategy']
  - Final shape: (45, 29)
  - Data preservation check: 45 rows maintained
Processing: SA_Campaign_List_20250908_20250908_XMoAbg.xlsx
  - Original shape: (46, 33)
  - After cleaning: (46, 32)
  - Extracted date: 2025-09-08 00:00:00
  - Dropped columns: ['Profile', 'Labels']
  - Excluded extra columns: ['Target type', 'Current Budget', 'SP Off-site Ads Strategy']
  - Final shape: (46, 29)
  - Data preservation check: 46 rows maintained
Processing: SA_Campaign_List_20250909_20250909_5FdPCK.xlsx
  - Original shape: (51, 33)
  - After cleaning: (51, 32)
  - Extracted date: 2025-09-09 00:00:00
  - Dropped columns: ['Profile', 'Labels']
  - Excluded extra columns: ['Target type', 'Current Budget', 'SP Off-site Ads Strategy']
  - Final shape: (51, 29)
  - Data preser

C:\Users\admin1\AppData\Local\Temp\ipykernel_23940\4164627072.py:509: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update(range_name, values)


Fallback append completed
Google Sheet update completed successfully!

Month 9 processing completed!


# SellerBoard (Daily)

In [1]:
import os
import json
import hashlib
import re
import pandas as pd
from datetime import datetime
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

class SBDataProcessor:
    def __init__(self, base_folder, credentials_path, sheet_id, worksheet_name):
        self.base_folder = base_folder
        self.credentials_path = credentials_path
        self.sheet_id = sheet_id
        self.worksheet_name = worksheet_name
        self.metadata_file = "sb_file_metadata.json"
        
        # Định nghĩa thứ tự cột chuẩn
        self.standard_columns = [
            'Product', 'ASIN', 'Date', 'SKU', 'Units', 'Refunds', 'Sales', 
            'Promo', 'Ads', 'Sponsored products (PPC)', '% Refunds', 'Refund сost',
            'Amazon fees', 'Cost of Goods', 'Gross profit', 'Net profit', 
            'Estimated payout', 'Real ACOS', 'Sessions', 'VAT', 'Shipping'
        ]
        
        # Initialize Google Sheets
        self._init_google_sheets()
        
        # Load existing metadata
        self.file_metadata = self._load_metadata()
        
    def _init_google_sheets(self):
        """Initialize Google Sheets connection"""
        scopes = ["https://www.googleapis.com/auth/spreadsheets", 
                  "https://www.googleapis.com/auth/drive"]
        creds = Credentials.from_service_account_file(self.credentials_path, scopes=scopes)
        self.client = gspread.authorize(creds)
        self.spreadsheet = self.client.open_by_key(self.sheet_id)
        self.worksheet = self.spreadsheet.worksheet(self.worksheet_name)
    
    def _load_metadata(self):
        """Load file metadata from JSON file"""
        if os.path.exists(self.metadata_file):
            with open(self.metadata_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}
    
    def _save_metadata(self):
        """Save file metadata to JSON file"""
        with open(self.metadata_file, 'w', encoding='utf-8') as f:
            json.dump(self.file_metadata, f, indent=2, ensure_ascii=False, default=str)
    
    def _get_file_hash(self, file_path):
        """Calculate file hash for change detection"""
        hash_md5 = hashlib.md5()
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()
    
    def extract_date_from_filename(self, filename):
        """Extract first DD_MM_YYYY pattern from filename"""
        match = re.search(r"(\d{2}_\d{2}_\d{4})", filename)
        if match:
            return datetime.strptime(match.group(1), "%d_%m_%Y").date()
        return None
    
    def _standardize_columns(self, df):
        """Standardize and select only required columns"""
        # Làm sạch tên cột
        df.columns = [str(c).strip() for c in df.columns]
        
        # Tạo mapping cho các tên cột có thể khác nhau
        column_mapping = {}
        df_columns_lower = [col.lower() for col in df.columns]
        
        for std_col in self.standard_columns:
            std_col_lower = std_col.lower()
            
            # Tìm cột khớp chính xác hoặc gần giống
            for i, df_col in enumerate(df.columns):
                df_col_lower = df_col.lower()
                
                # Khớp chính xác
                if std_col_lower == df_col_lower:
                    column_mapping[df_col] = std_col
                    break
                # Khớp một phần cho một số trường hợp đặc biệt
                elif 'sponsored' in std_col_lower and 'sponsored' in df_col_lower and 'ppc' in df_col_lower:
                    column_mapping[df_col] = std_col
                    break
                elif 'refund' in std_col_lower and 'cost' in std_col_lower and 'refund' in df_col_lower and ('cost' in df_col_lower or 'сost' in df_col_lower):
                    column_mapping[df_col] = std_col
                    break
        
        # Rename columns theo mapping
        df = df.rename(columns=column_mapping)
        
        # Chỉ giữ lại các cột cần thiết
        available_columns = [col for col in self.standard_columns if col in df.columns]
        df_filtered = df[available_columns].copy()
        
        # Thêm các cột thiếu với giá trị None
        for col in self.standard_columns:
            if col not in df_filtered.columns:
                df_filtered[col] = None
        
        # Sắp xếp lại theo thứ tự chuẩn
        df_filtered = df_filtered[self.standard_columns]
        
        print(f"📋 Available columns: {len(available_columns)}/{len(self.standard_columns)}")
        missing_cols = [col for col in self.standard_columns if col not in available_columns]
        if missing_cols:
            print(f"⚠️ Missing columns: {missing_cols}")
        
        return df_filtered
    
    def process_single_excel(self, file_path):
        """Process a single Excel file and return DataFrame with Date column"""
        try:
            df = pd.read_excel(file_path)
            df = df.dropna(axis=1, how="all")  
            
            # Extract date from filename
            date_val = self.extract_date_from_filename(os.path.basename(file_path))
            if date_val:
                df["Date"] = pd.to_datetime(date_val)
            
            # Standardize columns
            df = self._standardize_columns(df)
            
            return df
        except Exception as e:
            print(f"⚠️ Error processing {file_path}: {e}")
            return pd.DataFrame()
    
    def _is_july_august_file(self, file_date):
        """Check if file belongs to July or August"""
        if not file_date:
            return False
        return file_date.month in [7, 8, 9] and file_date.year == 2025  # Adjust year as needed
    
    def _should_process_file(self, file_path, file_date, is_initial_run=False):
        """Determine if file should be processed"""
        file_name = os.path.basename(file_path)
        current_hash = self._get_file_hash(file_path)
        modification_time = os.path.getmtime(file_path)
        
        # For initial run, process all July-August files
        if is_initial_run:
            if self._is_july_august_file(file_date):
                print(f"🔄 Initial run: Processing July/August file {file_name}")
                return True
            return False
        
        # For subsequent runs, check if file is new or changed
        if file_name not in self.file_metadata:
            print(f"➕ New file detected: {file_name}")
            return True
        
        stored_metadata = self.file_metadata[file_name]
        
        # Check if file has been modified (hash changed or modification time changed)
        if (stored_metadata.get('hash') != current_hash or 
            stored_metadata.get('modification_time') != modification_time):
            print(f"🔄 Modified file detected: {file_name}")
            return True
        
        print(f"⏭️ Skipping unchanged file: {file_name}")
        return False
    
    def _update_file_metadata(self, file_path, file_date):
        """Update metadata for processed file"""
        file_name = os.path.basename(file_path)
        self.file_metadata[file_name] = {
            'path': file_path,
            'date': file_date,
            'hash': self._get_file_hash(file_path),
            'modification_time': os.path.getmtime(file_path),
            'processed_at': datetime.now()
        }
    
    def process_files(self, initial_run=False):
        """
        Main processing function
        Args:
            initial_run (bool): If True, reprocess all July-August files from scratch
        """
        print("=" * 60)
        if initial_run:
            print("🚀 INITIAL RUN: Processing all July-August files")
            # Clear existing July-August metadata for fresh start
            files_to_remove = []
            for file_name, metadata in self.file_metadata.items():
                if isinstance(metadata.get('date'), str):
                    file_date = datetime.strptime(metadata['date'], "%Y-%m-%d").date()
                elif metadata.get('date'):
                    file_date = metadata['date']
                else:
                    continue
                    
                if self._is_july_august_file(file_date):
                    files_to_remove.append(file_name)
            
            for file_name in files_to_remove:
                del self.file_metadata[file_name]
                print(f"🗑️ Cleared metadata for July/August file: {file_name}")
        else:
            print("🔄 INCREMENTAL RUN: Processing new/modified files only")
        print("=" * 60)
        
        all_dataframes = []
        processed_files = []
        
        # Scan all Excel files in subfolders
        for root, dirs, files in os.walk(self.base_folder):
            for file in files:
                if file.endswith(".xlsx"):
                    file_path = os.path.join(root, file)
                    file_date = self.extract_date_from_filename(file)
                    
                    if self._should_process_file(file_path, file_date, initial_run):
                        print(f"📊 Processing: {file}")
                        df = self.process_single_excel(file_path)
                        
                        if not df.empty:
                            all_dataframes.append(df)
                            processed_files.append(file)
                            self._update_file_metadata(file_path, file_date)
                        else:
                            print(f"⚠️ Empty dataframe for: {file}")
        
        # Combine all processed data
        if all_dataframes:
            print(f"\n📈 Combining {len(all_dataframes)} dataframes...")
            master_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
            
            # Sort by date, then by sales (descending)
            if "Date" in master_df.columns and "Sales" in master_df.columns:
                master_df = master_df.sort_values(["Date", "Sales"], ascending=[True, False])
            elif "Date" in master_df.columns:
                master_df = master_df.sort_values("Date", ascending=True)
            
            print(f"✅ Combined data shape: {master_df.shape}")
            if "Date" in master_df.columns:
                print(f"📅 Date range: {master_df['Date'].min()} to {master_df['Date'].max()}")
            
            # Upload to Google Sheets
            self._upload_to_sheets(master_df)
            
            # Save metadata
            self._save_metadata()
            
            print(f"\n🎉 Successfully processed {len(processed_files)} files:")
            for file in processed_files:
                print(f"   ✓ {file}")
            
            return master_df
        else:
            print("ℹ️ No files to process.")
            return pd.DataFrame()
    
    def _upload_to_sheets(self, df):
        """Upload DataFrame to Google Sheets"""
        try:
            print("📤 Uploading to Google Sheets...")
            
            # Clear existing data (columns A to U to match our 21 standard columns)
            self.worksheet.batch_clear(['A:U'])
            
            # Upload new data
            set_with_dataframe(self.worksheet, df)
            
            print(f"✅ Successfully uploaded {len(df)} rows to Google Sheets")
            print(f"🔗 Sheet: {self.worksheet_name}")
            print(f"📋 Columns: {', '.join(self.standard_columns)}")
            
        except Exception as e:
            print(f"❌ Error uploading to Google Sheets: {e}")
    
    def get_processing_summary(self):
        """Get summary of processed files"""
        if not self.file_metadata:
            return "No files processed yet."
        
        july_files = []
        august_files = []
        other_files = []
        
        for file_name, metadata in self.file_metadata.items():
            if isinstance(metadata.get('date'), str):
                file_date = datetime.strptime(metadata['date'], "%Y-%m-%d").date()
            elif metadata.get('date'):
                file_date = metadata['date']
            else:
                other_files.append(file_name)
                continue
            
            if file_date.month == 7:
                july_files.append(file_name)
            elif file_date.month == 8:
                august_files.append(file_name)
            else:
                other_files.append(file_name)
        
        summary = f"""
📊 PROCESSING SUMMARY
=====================
July files: {len(july_files)}
August files: {len(august_files)}
Other files: {len(other_files)}
Total files: {len(self.file_metadata)}
Standard columns: {len(self.standard_columns)}
Last run: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        """
        return summary

# Usage Example
if __name__ == "__main__":
    # Configuration
    config = {
        'base_folder': "C:/Users/admin1/Desktop/Performance-Tracking/Agg-SB/H2_2025_US",
        'credentials_path': "c:/Users/admin1/Downloads/new_credential.json",
        'sheet_id': "1lZ4dsi94HaeWshsEizKTyNHeOOG0tpLJhzL9pMxvd6k",
        'worksheet_name': "Raw_SB_H2_2025_US"
    }
    
    # Initialize processor
    processor = SBDataProcessor(**config)
    
    # First time: Run with initial_run=True to reprocess all July-August files
    print("Choose run mode:")
    print("1. Initial run (reprocess all July-August files)")
    print("2. Incremental run (process only new/modified files)")
    
    choice = input("Enter choice (1 or 2): ").strip()
    
    if choice == "1":
        result_df = processor.process_files(initial_run=True)
    else:
        result_df = processor.process_files(initial_run=False)
    
    # Print summary
    print(processor.get_processing_summary())
    
    # Show column info
    print(f"\n📋 Standard columns ({len(processor.standard_columns)}):")
    for i, col in enumerate(processor.standard_columns, 1):
        print(f"   {i:2d}. {col}")

Choose run mode:
1. Initial run (reprocess all July-August files)
2. Incremental run (process only new/modified files)
🚀 INITIAL RUN: Processing all July-August files
🔄 Initial run: Processing July/August file NewEleven_Dashboard Products Group by ASIN_01_07_2025-01_07_2025_(02_16_59_866).xlsx
📊 Processing: NewEleven_Dashboard Products Group by ASIN_01_07_2025-01_07_2025_(02_16_59_866).xlsx
📋 Available columns: 19/21
⚠️ Missing columns: ['VAT', 'Shipping']
🔄 Initial run: Processing July/August file NewEleven_Dashboard Products Group by ASIN_02_07_2025-02_07_2025_(02_17_19_401).xlsx
📊 Processing: NewEleven_Dashboard Products Group by ASIN_02_07_2025-02_07_2025_(02_17_19_401).xlsx
📋 Available columns: 19/21
⚠️ Missing columns: ['VAT', 'Shipping']
🔄 Initial run: Processing July/August file NewEleven_Dashboard Products Group by ASIN_03_07_2025-03_07_2025_(02_17_35_203).xlsx
📊 Processing: NewEleven_Dashboard Products Group by ASIN_03_07_2025-03_07_2025_(02_17_35_203).xlsx
📋 Available column

C:\Users\admin1\AppData\Local\Temp\ipykernel_21800\935954928.py:237: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  master_df = pd.concat(all_dataframes, ignore_index=True, sort=False)


✅ Successfully uploaded 15148 rows to Google Sheets
🔗 Sheet: Raw_SB_H2_2025_US
📋 Columns: Product, ASIN, Date, SKU, Units, Refunds, Sales, Promo, Ads, Sponsored products (PPC), % Refunds, Refund сost, Amazon fees, Cost of Goods, Gross profit, Net profit, Estimated payout, Real ACOS, Sessions, VAT, Shipping

🎉 Successfully processed 64 files:
   ✓ NewEleven_Dashboard Products Group by ASIN_01_07_2025-01_07_2025_(02_16_59_866).xlsx
   ✓ NewEleven_Dashboard Products Group by ASIN_02_07_2025-02_07_2025_(02_17_19_401).xlsx
   ✓ NewEleven_Dashboard Products Group by ASIN_03_07_2025-03_07_2025_(02_17_35_203).xlsx
   ✓ NewEleven_Dashboard Products Group by ASIN_04_07_2025-04_07_2025_(02_17_52_472).xlsx
   ✓ NewEleven_Dashboard Products Group by ASIN_05_07_2025-05_07_2025_(02_18_20_680).xlsx
   ✓ NewEleven_Dashboard Products Group by ASIN_06_07_2025-06_07_2025_(02_18_35_117).xlsx
   ✓ NewEleven_Dashboard Products Group by ASIN_07_07_2025-07_07_2025_(02_18_54_289).xlsx
   ✓ NewEleven_Dashboard P

# SellerBoard Tháng 9

In [2]:
import os
import json
import hashlib
import re
import pandas as pd
from datetime import datetime
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

class SBSeptemberProcessor:
    def __init__(self, base_folder, credentials_path, sheet_id, worksheet_name):
        self.base_folder = base_folder
        self.credentials_path = credentials_path
        self.sheet_id = sheet_id
        self.worksheet_name = worksheet_name
        self.metadata_file = "sb_september_metadata.json"
        
        # Định nghĩa thứ tự cột chuẩn
        self.standard_columns = [
            'Product', 'ASIN', 'Date', 'SKU', 'Units', 'Refunds', 'Sales', 
            'Promo', 'Ads', 'Sponsored products (PPC)', '% Refunds', 'Refund сost',
            'Amazon fees', 'Cost of Goods', 'Gross profit', 'Net profit', 
            'Estimated payout', 'Real ACOS', 'Sessions', 'VAT', 'Shipping'
        ]
        
        # Initialize Google Sheets
        self._init_google_sheets()
        
        # Load existing metadata cho tháng 9
        self.file_metadata = self._load_metadata()
        
    def _init_google_sheets(self):
        """Initialize Google Sheets connection"""
        scopes = ["https://www.googleapis.com/auth/spreadsheets", 
                  "https://www.googleapis.com/auth/drive"]
        creds = Credentials.from_service_account_file(self.credentials_path, scopes=scopes)
        self.client = gspread.authorize(creds)
        self.spreadsheet = self.client.open_by_key(self.sheet_id)
        self.worksheet = self.spreadsheet.worksheet(self.worksheet_name)
    
    def _load_metadata(self):
        """Load file metadata from JSON file"""
        if os.path.exists(self.metadata_file):
            with open(self.metadata_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}
    
    def _save_metadata(self):
        """Save file metadata to JSON file"""
        with open(self.metadata_file, 'w', encoding='utf-8') as f:
            json.dump(self.file_metadata, f, indent=2, ensure_ascii=False, default=str)
    
    def _get_file_hash(self, file_path):
        """Calculate file hash for change detection"""
        hash_md5 = hashlib.md5()
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()
    
    def extract_date_from_filename(self, filename):
        """Extract first DD_MM_YYYY pattern from filename"""
        match = re.search(r"(\d{2}_\d{2}_\d{4})", filename)
        if match:
            return datetime.strptime(match.group(1), "%d_%m_%Y").date()
        return None
    
    def _standardize_columns(self, df):
        """Standardize and select only required columns"""
        df.columns = [str(c).strip() for c in df.columns]

        column_mapping = {}
        for std_col in self.standard_columns:
            std_col_lower = std_col.lower()
            for df_col in df.columns:
                df_col_lower = df_col.lower()
                if std_col_lower == df_col_lower:
                    column_mapping[df_col] = std_col
                    break
                elif 'sponsored' in std_col_lower and 'sponsored' in df_col_lower and 'ppc' in df_col_lower:
                    column_mapping[df_col] = std_col
                    break
                elif 'refund' in std_col_lower and 'cost' in std_col_lower and 'refund' in df_col_lower and ('cost' in df_col_lower or 'сost' in df_col_lower):
                    column_mapping[df_col] = std_col
                    break

        df = df.rename(columns=column_mapping)
        available_columns = [col for col in self.standard_columns if col in df.columns]
        df_filtered = df[available_columns].copy()

        # Thêm các cột thiếu
        for col in self.standard_columns:
            if col not in df_filtered.columns:
                df_filtered[col] = pd.NA

        # Sắp xếp đúng thứ tự chuẩn
        df_filtered = df_filtered[self.standard_columns]

        return df_filtered
    
    def process_single_excel(self, file_path):
        """Process a single Excel file and return DataFrame with Date column"""
        try:
            df = pd.read_excel(file_path)
            df = df.dropna(axis=1, how="all")  
            
            # Extract date from filename
            date_val = self.extract_date_from_filename(os.path.basename(file_path))
            if date_val:
                df["Date"] = pd.to_datetime(date_val)
            
            # Standardize columns
            df = self._standardize_columns(df)
            
            return df
        except Exception as e:
            print(f"⚠️ Error processing {file_path}: {e}")
            return pd.DataFrame()
    
    def _is_september_file(self, file_date):
        """Check if file belongs to September 2025"""
        if not file_date:
            return False
        return file_date.month == 9 and file_date.year == 2025
    
    def _should_process_file(self, file_path, file_date):
        """Determine if September file should be processed"""
        # Chỉ xử lý file tháng 9
        if not self._is_september_file(file_date):
            return False
            
        file_name = os.path.basename(file_path)
        current_hash = self._get_file_hash(file_path)
        modification_time = os.path.getmtime(file_path)
        
        # Check if file is new or changed
        if file_name not in self.file_metadata:
            print(f"➕ New September file detected: {file_name}")
            return True
        
        stored_metadata = self.file_metadata[file_name]
        
        # Check if file has been modified
        if (stored_metadata.get('hash') != current_hash or 
            stored_metadata.get('modification_time') != modification_time):
            print(f"🔄 Modified September file detected: {file_name}")
            return True
        
        print(f"⏭️ Skipping unchanged September file: {file_name}")
        return False
    
    def _update_file_metadata(self, file_path, file_date):
        """Update metadata for processed file"""
        file_name = os.path.basename(file_path)
        self.file_metadata[file_name] = {
            'path': file_path,
            'date': file_date,
            'hash': self._get_file_hash(file_path),
            'modification_time': os.path.getmtime(file_path),
            'processed_at': datetime.now()
        }
    
    def get_existing_sheet_data_count(self):
        """Get current number of rows in the sheet"""
        try:
            # Lấy tất cả giá trị trong cột A để đếm số dòng có dữ liệu
            all_values = self.worksheet.col_values(1)  # Column A
            # Trừ đi header row
            data_rows = len([val for val in all_values if val.strip()]) - 1 if all_values else 0
            print(f"📊 Current data rows in sheet: {data_rows}")
            return data_rows
        except Exception as e:
            print(f"⚠️ Error getting sheet data count: {e}")
            return 0
    
    def process_september_files(self):
        print("=" * 60)
        print("🗓️ SEPTEMBER PROCESSOR: Processing September 2025 files only")
        print("=" * 60)

        existing_rows = self.get_existing_sheet_data_count()
        all_dataframes, processed_files = [], []

        for root, dirs, files in os.walk(self.base_folder):
            for file in files:
                if file.endswith(".xlsx"):
                    file_path = os.path.join(root, file)
                    file_date = self.extract_date_from_filename(file)

                    if self._should_process_file(file_path, file_date):
                        df = self.process_single_excel(file_path)
                        if not df.empty:
                            all_dataframes.append(df)
                            processed_files.append(file)
                            self._update_file_metadata(file_path, file_date)

        if all_dataframes:
            september_df = pd.concat(all_dataframes, ignore_index=True, sort=False)

            # Sort by Date then Sales
            if "Date" in september_df.columns and "Sales" in september_df.columns:
                september_df = september_df.sort_values(["Date", "Sales"], ascending=[True, False])
            elif "Date" in september_df.columns:
                september_df = september_df.sort_values("Date")

            # Đảm bảo cột theo đúng thứ tự chuẩn
            september_df = september_df[self.standard_columns]

            self._append_to_sheets(september_df, existing_rows)
            self._save_metadata()
            return september_df
        else:
            print("ℹ️ No new September files to process.")
            return pd.DataFrame()
    
    def _append_to_sheets(self, df, existing_rows):
        try:
            print("📤 Appending September data to Google Sheets...")
            start_row = existing_rows + 2

            values_to_append = []
            for _, row in df.iterrows():
                row_values = []
                for col in self.standard_columns:
                    val = row[col]
                    if pd.isna(val):
                        row_values.append("")  # để Sheets giữ trống
                    elif isinstance(val, (pd.Timestamp, datetime)):
                        row_values.append(val.strftime("%Y-%m-%d"))
                    else:
                        row_values.append(val)
                values_to_append.append(row_values)

            end_col = chr(ord('A') + len(self.standard_columns) - 1)
            end_row = start_row + len(df) - 1
            range_name = f"A{start_row}:{end_col}{end_row}"

            self.worksheet.update(range_name, values_to_append)
            print(f"✅ Successfully appended {len(df)} rows to Google Sheets")
        except Exception as e:
            print(f"❌ Error appending to Google Sheets: {e}")
    
    def get_processing_summary(self):
        """Get summary of processed September files"""
        if not self.file_metadata:
            return "No September files processed yet."
        
        september_files = []
        
        for file_name, metadata in self.file_metadata.items():
            if isinstance(metadata.get('date'), str):
                file_date = datetime.strptime(metadata['date'], "%Y-%m-%d").date()
            elif metadata.get('date'):
                file_date = metadata['date']
            else:
                continue
            
            if file_date.month == 9 and file_date.year == 2025:
                september_files.append(file_name)
        
        summary = f"""
📊 SEPTEMBER PROCESSING SUMMARY
===============================
September 2025 files: {len(september_files)}
Standard columns: {len(self.standard_columns)}
Last run: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

September files processed:
{chr(10).join([f"  • {f}" for f in september_files]) if september_files else "  None"}
        """
        return summary

# Usage Example
if __name__ == "__main__":
    # Configuration
    config = {
        'base_folder': "C:/Users/admin1/Desktop/Performance-Tracking/Agg-SB/H2_2025_CA",
        'credentials_path': "c:/Users/admin1/Downloads/new_credential.json",
        'sheet_id': "1lZ4dsi94HaeWshsEizKTyNHeOOG0tpLJhzL9pMxvd6k",
        'worksheet_name': "Raw_SB_H2_2025_CA"
    }
    
    # Initialize September processor
    processor = SBSeptemberProcessor(**config)
    
    print("🗓️ September Data Processor")
    print("This will process ONLY September 2025 files and append to existing sheet data")
    
    confirm = input("Continue? (y/n): ").strip().lower()
    
    if confirm == 'y':
        # Process September files
        result_df = processor.process_september_files()
        
        # Print summary
        print(processor.get_processing_summary())
        
        # Show column info
        print(f"\n📋 Standard columns ({len(processor.standard_columns)}):")
        for i, col in enumerate(processor.standard_columns, 1):
            print(f"   {i:2d}. {col}")
    else:
        print("❌ Operation cancelled")

🗓️ September Data Processor
This will process ONLY September 2025 files and append to existing sheet data
🗓️ SEPTEMBER PROCESSOR: Processing September 2025 files only
📊 Current data rows in sheet: 1912
➕ New September file detected: NewEleven_Dashboard Products Group by ASIN_01_09_2025-01_09_2025_(00_20_30_218).xlsx
➕ New September file detected: NewEleven_Dashboard Products Group by ASIN_02_09_2025-02_09_2025_(00_20_43_104).xlsx
➕ New September file detected: NewEleven_Dashboard Products Group by ASIN_03_09_2025-03_09_2025_(00_20_55_443).xlsx
➕ New September file detected: NewEleven_Dashboard Products Group by ASIN_04_09_2025-04_09_2025_(00_21_08_726).xlsx
➕ New September file detected: NewEleven_Dashboard Products Group by ASIN_05_09_2025-05_09_2025_(00_21_20_133).xlsx
➕ New September file detected: NewEleven_Dashboard Products Group by ASIN_06_09_2025-06_09_2025_(00_21_33_587).xlsx
➕ New September file detected: NewEleven_Dashboard Products Group by ASIN_07_09_2025-07_09_2025_(00_21

C:\Users\admin1\AppData\Local\Temp\ipykernel_23940\2261602916.py:199: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  september_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
C:\Users\admin1\AppData\Local\Temp\ipykernel_23940\2261602916.py:239: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  self.worksheet.update(range_name, values_to_append)


✅ Successfully appended 249 rows to Google Sheets

📊 SEPTEMBER PROCESSING SUMMARY
September 2025 files: 9
Standard columns: 21
Last run: 2025-09-10 14:25:44

September files processed:
  • NewEleven_Dashboard Products Group by ASIN_01_09_2025-01_09_2025_(00_20_30_218).xlsx
  • NewEleven_Dashboard Products Group by ASIN_02_09_2025-02_09_2025_(00_20_43_104).xlsx
  • NewEleven_Dashboard Products Group by ASIN_03_09_2025-03_09_2025_(00_20_55_443).xlsx
  • NewEleven_Dashboard Products Group by ASIN_04_09_2025-04_09_2025_(00_21_08_726).xlsx
  • NewEleven_Dashboard Products Group by ASIN_05_09_2025-05_09_2025_(00_21_20_133).xlsx
  • NewEleven_Dashboard Products Group by ASIN_06_09_2025-06_09_2025_(00_21_33_587).xlsx
  • NewEleven_Dashboard Products Group by ASIN_07_09_2025-07_09_2025_(00_21_45_630).xlsx
  • NewEleven_Dashboard Products Group by ASIN_08_09_2025-08_09_2025_(00_21_57_464).xlsx
  • NewEleven_Dashboard Products Group by ASIN_09_09_2025-09_09_2025_(00_22_07_850).xlsx
        

📋 St

# XNurta H2 2024 (Daily)

In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import glob
import re
import json
import hashlib
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

class XNurtaDataProcessor2024:
    def __init__(self, base_folder, credentials_path, sheet_id, worksheet_name):
        self.base_folder = base_folder
        self.credentials_path = credentials_path
        self.sheet_id = sheet_id
        self.worksheet_name = worksheet_name
        self.metadata_file = "xnurta_file_metadata_2024.json"
        
        # Initialize Google Sheets
        self._init_google_sheets()
        
        # Load existing metadata
        self.file_metadata = self._load_metadata()
        
    def _init_google_sheets(self):
        """Initialize Google Sheets connection"""
        scopes = ["https://www.googleapis.com/auth/spreadsheets", 
                  "https://www.googleapis.com/auth/drive"]
        creds = Credentials.from_service_account_file(self.credentials_path, scopes=scopes)
        self.client = gspread.authorize(creds)
        self.spreadsheet = self.client.open_by_key(self.sheet_id)
        self.worksheet = self.spreadsheet.worksheet(self.worksheet_name)
    
    def _load_metadata(self):
        """Load file metadata from JSON file"""
        if os.path.exists(self.metadata_file):
            with open(self.metadata_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}
    
    def _save_metadata(self):
        """Save file metadata to JSON file"""
        with open(self.metadata_file, 'w', encoding='utf-8') as f:
            json.dump(self.file_metadata, f, indent=2, ensure_ascii=False, default=str)
    
    def _get_file_hash(self, file_path):
        """Calculate file hash for change detection"""
        hash_md5 = hashlib.md5()
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()
        
    def extract_date_from_filename(self, filename):
        """
        Extract date from filename pattern: SA_Campaign_List_YYYYMMDD_YYYYMMDD_hash.xlsx
        Returns the first date (start date)
        """
        pattern = r'SA_Campaign_List_(\d{8})_\d{8}_.*\.xlsx'
        match = re.search(pattern, os.path.basename(filename))
        if match:
            date_str = match.group(1)
            return pd.to_datetime(date_str, format='%Y%m%d')
        return None
        
    def clean_currency_column(self, column):
        """Remove $ symbol and convert to float"""
        if column.dtype == 'object':
            cleaned = column.astype(str).str.replace(r'[$,]', '', regex=True)
            cleaned = cleaned.replace(['', 'nan', 'NaN'], np.nan)
            return pd.to_numeric(cleaned, errors='coerce')
        return column
        
    def convert_to_float(self, column):
        """Convert object columns to float"""
        if column.dtype == 'object':
            cleaned = column.astype(str).str.replace(r'[%,]', '', regex=True)
            cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
            return pd.to_numeric(cleaned, errors='coerce')
        return column
        
    def convert_to_int(self, column):
        """Convert object columns to int"""
        if column.dtype == 'object':
            cleaned = column.astype(str).str.replace(r'[,]', '', regex=True)
            cleaned = cleaned.replace(['', 'nan', 'NaN', '--', 'N/A'], np.nan)
            float_col = pd.to_numeric(cleaned, errors='coerce')
            return float_col.astype('Int64')  # Nullable integer type
        return column
        
    def extract_asin_from_portfolio(self, portfolio_str):
        """Extract ASIN from Portfolio string"""
        if pd.isna(portfolio_str) or portfolio_str == '':
            return None
        
        portfolio_str = str(portfolio_str)
        
        # Pattern 1: B + 9 alphanumeric (most common ASIN format)
        pattern1 = r'B[A-Z0-9]{9}'
        match1 = re.search(pattern1, portfolio_str)
        if match1:
            return match1.group()
        
        # Pattern 2: 10 alphanumeric characters starting with letter
        pattern2 = r'[A-Z][A-Z0-9]{9}'
        match2 = re.search(pattern2, portfolio_str)
        if match2:
            return match2.group()
        
        # Pattern 3: Any 10 consecutive alphanumeric characters
        pattern3 = r'[A-Z0-9]{10}'
        match3 = re.search(pattern3, portfolio_str)
        if match3:
            return match3.group()
        
        # Pattern 4: 10 alphanumeric with possible lowercase (convert to uppercase)
        pattern4 = r'[A-Za-z0-9]{10}'
        match4 = re.search(pattern4, portfolio_str)
        if match4:
            return match4.group().upper()
        
        # If no pattern matches, return first 10 characters as fallback
        clean_str = re.sub(r'[^A-Za-z0-9]', '', portfolio_str)
        if len(clean_str) >= 10:
            return clean_str[:10].upper()
        
        return portfolio_str[:10] if len(portfolio_str) >= 10 else portfolio_str
        
    def normalize_campaign_types(self, text):
        """Normalize campaign type keywords"""
        if pd.isna(text) or text == '':
            return text
        
        text = str(text)
        
        normalizations = {
            'sponsoredBrands': 'SB',
            'sponsoredDisplay': 'SD', 
            'sponsoredProducts': 'SP',
            'sponsoredbrands': 'SB',
            'sponsoreddisplay': 'SD',
            'sponsoredproducts': 'SP',
            'Sponsored Brands': 'SB',
            'Sponsored Display': 'SD',
            'Sponsored Products': 'SP'
        }
        
        for original, normalized in normalizations.items():
            text = text.replace(original, normalized)
        
        return text
        
    def process_single_excel(self, file_path):
        """Process a single Excel file according to specifications"""
        try:
            # Read Excel file
            df = pd.read_excel(file_path)
            
            # Extract date from filename
            date_extracted = self.extract_date_from_filename(file_path)
            
            # Drop specified columns if they exist
            columns_to_drop = [
                'Profile', 
                'Labels', 
                'Budget group',
                'Status',
                'Current Budget',
                'SP Off-site Ads Strategy',
                'Bidding Strategy',
                'Sales Same SKU',
                'Sales Other SKU',
                'Orders Same SKU',
                'Orders Other SKU',
                'Units Same SKU',
                'Units Other SKU'
            ]
            existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
            if existing_columns_to_drop:
                df = df.drop(columns=existing_columns_to_drop)
                print(f"   🗑️ Dropped columns: {', '.join(existing_columns_to_drop)}")
            
            # Add ASIN column as first column (extract ASIN from Portfolio)
            if 'Portfolio' in df.columns:
                df.insert(0, 'ASIN', df['Portfolio'].apply(self.extract_asin_from_portfolio))
            
            # Add Date column
            df.insert(1, 'Date', date_extracted)
            
            # Normalize campaign types in Campaign Type column
            if 'Campaign type' in df.columns:
                df['Campaign type'] = df['Campaign type'].apply(self.normalize_campaign_types)
            
            # Clean currency columns
            currency_columns = ['Daily Budget']
            for col in currency_columns:
                if col in df.columns:
                    df[col] = self.clean_currency_column(df[col])
            
            # Convert specified columns to float
            float_columns = ['Avg.time in Budget', 'Top-of-search IS', 'CPC', 'CVR', 'ACOS', 'ROAS']
            for col in float_columns:
                if col in df.columns:
                    df[col] = self.convert_to_float(df[col])
            
            # Note: Removed int_columns conversion since those columns are now dropped
            
            print(f"✅ Successfully processed: {os.path.basename(file_path)}")
            return df
            
        except Exception as e:
            print(f"⚠️ Error processing {file_path}: {str(e)}")
            return pd.DataFrame()
            
    def _is_h2_2024_file(self, file_date):
        """Check if file belongs to Q3 2024 (July to September 2024)"""
        if not file_date:
            return False
        return file_date.month in [7, 8, 9] and file_date.year == 2024
        
    def _should_process_file(self, file_path, file_date, is_initial_run=False):
        """Determine if file should be processed"""
        file_name = os.path.basename(file_path)
        current_hash = self._get_file_hash(file_path)
        modification_time = os.path.getmtime(file_path)
        
        # For initial run, process all H2 2024 files
        if is_initial_run:
            if self._is_h2_2024_file(file_date):
                print(f"🔄 Initial run: Processing Q3 2024 file {file_name}")
                return True
            return False
        
        # For subsequent runs, check if file is new or changed
        if file_name not in self.file_metadata:
            print(f"➕ New file detected: {file_name}")
            return True
        
        stored_metadata = self.file_metadata[file_name]
        
        if (stored_metadata.get('hash') != current_hash or 
            stored_metadata.get('modification_time') != modification_time):
            print(f"🔄 Modified file detected: {file_name}")
            return True
        
        print(f"⏭️ Skipping unchanged file: {file_name}")
        return False
        
    def _update_file_metadata(self, file_path, file_date):
        """Update metadata for processed file"""
        file_name = os.path.basename(file_path)
        self.file_metadata[file_name] = {
            'path': file_path,
            'date': file_date,
            'hash': self._get_file_hash(file_path),
            'modification_time': os.path.getmtime(file_path),
            'processed_at': datetime.now()
        }
        
    def process_files(self, initial_run=False):
        """Main processing function for XNurta 2024 data"""
        print("=" * 60)
        if initial_run:
            print("🚀 INITIAL RUN: Processing Q3 2024 XNurta files (Jul-Sep)")
            # Clear existing Q3 2024 metadata for fresh start
            files_to_remove = []
            for file_name, metadata in self.file_metadata.items():
                if isinstance(metadata.get('date'), str):
                    file_date = pd.to_datetime(metadata['date'])
                elif metadata.get('date'):
                    file_date = metadata['date']
                else:
                    continue
                    
                if self._is_h2_2024_file(file_date):
                    files_to_remove.append(file_name)
            
            for file_name in files_to_remove:
                del self.file_metadata[file_name]
                print(f"🗑️ Cleared metadata for Q3 2024 file: {file_name}")
        else:
            print("🔄 INCREMENTAL RUN: Processing new/modified files only")
        print("=" * 60)
        
        all_dataframes = []
        processed_files = []
        
        # Scan all Excel files in subfolders (Tháng 7, Tháng 8, etc.)
        for root, dirs, files in os.walk(self.base_folder):
            for file in files:
                if file.endswith(".xlsx"):
                    file_path = os.path.join(root, file)
                    file_date = self.extract_date_from_filename(file)
                    
                    if self._should_process_file(file_path, file_date, initial_run):
                        print(f"📊 Processing: {file}")
                        df = self.process_single_excel(file_path)
                        
                        if not df.empty:
                            all_dataframes.append(df)
                            processed_files.append(file)
                            self._update_file_metadata(file_path, file_date)
                        else:
                            print(f"⚠️ Empty dataframe for: {file}")
        
        # Combine all processed data
        if all_dataframes:
            print(f"\n📈 Combining {len(all_dataframes)} dataframes...")
            master_df = pd.concat(all_dataframes, ignore_index=True, sort=False)
            
            # Sort by Date and ASIN
            if "Date" in master_df.columns:
                master_df = master_df.sort_values(['Date', 'ASIN'], na_position='last')
            
            # Reset index
            master_df = master_df.reset_index(drop=True)
            
            print(f"✅ Combined data shape: {master_df.shape}")
            print(f"📅 Date range: {master_df['Date'].min()} to {master_df['Date'].max()}")
            
            # Upload to Google Sheets
            self._upload_to_sheets(master_df)
            
            # Save metadata
            self._save_metadata()
            
            print(f"\n🎉 Successfully processed {len(processed_files)} files:")
            for file in processed_files:
                print(f"   ✓ {file}")
            
            return master_df
        else:
            print("ℹ️ No files to process.")
            return pd.DataFrame()
            
    def _upload_to_sheets(self, df):
        """Upload DataFrame to Google Sheets"""
        try:
            print("📤 Uploading to Google Sheets...")
            
            # Clear limited columns range (A to AZ) instead of entire sheet
            self.worksheet.batch_clear(['A:AZ'])
            
            # Upload new data
            set_with_dataframe(self.worksheet, df)
            
            print(f"✅ Successfully uploaded {len(df)} rows to Google Sheets")
            print(f"🔗 Sheet: {self.worksheet_name}")
            
        except Exception as e:
            print(f"❌ Error uploading to Google Sheets: {e}")
            
    def get_processing_summary(self):
        """Get summary of processed files by month"""
        if not self.file_metadata:
            return "No files processed yet."
        
        monthly_files = {
            7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 'other': []
        }
        month_names = {
            7: 'July', 8: 'August', 9: 'September', 
            10: 'October', 11: 'November', 12: 'December'
        }
        
        for file_name, metadata in self.file_metadata.items():
            if isinstance(metadata.get('date'), str):
                file_date = pd.to_datetime(metadata['date'])
            elif metadata.get('date'):
                file_date = metadata['date']
            else:
                monthly_files['other'].append(file_name)
                continue
            
            if file_date.month in monthly_files:
                monthly_files[file_date.month].append(file_name)
            else:
                monthly_files['other'].append(file_name)
        
        summary = f"""
📊 PROCESSING SUMMARY - XNurta 2024 Q3
======================================="""
        
        for month_num in [7, 8, 9]:
            count = len(monthly_files[month_num])
            summary += f"\n{month_names[month_num]} files: {count}"
        
        summary += f"""
Other files: {len(monthly_files['other'])}
Total files: {len(self.file_metadata)}
Last run: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
        """
        return summary

# Usage Example
if __name__ == "__main__":
    # Configuration for XNurta 2024
    config = {
        'base_folder': "C:/Users/admin1/Desktop/Performance-Tracking/Xnurta 2024 (by day)",  # Update path
        'credentials_path': "c:/Users/admin1/Downloads/new_credential.json",
        'sheet_id': "1lZ4dsi94HaeWshsEizKTyNHeOOG0tpLJhzL9pMxvd6k",
        'worksheet_name': "Raw_XNurta_H2_2024"
    }
    
    # Initialize processor
    processor = XNurtaDataProcessor2024(**config)
    
    # Choose run mode
    print("Choose run mode for XNurta 2024 Q3 data:")
    print("1. Initial run (reprocess all Q3 2024 files: Jul-Sep)")
    print("2. Incremental run (process only new/modified files)")
    
    choice = input("Enter choice (1 or 2): ").strip()
    
    if choice == "1":
        result_df = processor.process_files(initial_run=True)
    else:
        result_df = processor.process_files(initial_run=False)
    
    # Print summary
    print(processor.get_processing_summary())

Choose run mode for XNurta 2024 Q3 data:
1. Initial run (reprocess all Q3 2024 files: Jul-Sep)
2. Incremental run (process only new/modified files)
🚀 INITIAL RUN: Processing Q3 2024 XNurta files (Jul-Sep)
🗑️ Cleared metadata for Q3 2024 file: SA_Campaign_List_20240701_20240701_38C2RG.xlsx
🗑️ Cleared metadata for Q3 2024 file: SA_Campaign_List_20240702_20240702_9TkpbM.xlsx
🗑️ Cleared metadata for Q3 2024 file: SA_Campaign_List_20240703_20240703_G80TRn.xlsx
🗑️ Cleared metadata for Q3 2024 file: SA_Campaign_List_20240704_20240704_gZtCIL.xlsx
🗑️ Cleared metadata for Q3 2024 file: SA_Campaign_List_20240705_20240705_A1i8Dn.xlsx
🗑️ Cleared metadata for Q3 2024 file: SA_Campaign_List_20240706_20240706_ed4ep2.xlsx
🗑️ Cleared metadata for Q3 2024 file: SA_Campaign_List_20240707_20240707_1nbAjc.xlsx
🗑️ Cleared metadata for Q3 2024 file: SA_Campaign_List_20240708_20240708_nSsf7X.xlsx
🗑️ Cleared metadata for Q3 2024 file: SA_Campaign_List_20240709_20240709_Sa3Pwo.xlsx
🗑️ Cleared metadata for Q3 202

# Version 2